# Support

If you have any questions or want to report an issue regarding the Python SDK please email <a href="mailto:clarion-support@coremont.com?subject=Clarion API/SDK Support Query&body=Your Email Account for the API (if not your own):%0D%0ADate/time of the issue:%0D%0ARelevant section of the log file:%0D%0ASDK Version:%0D%0AWhat you expected to happen and what actually happened:%0D%0AAny other relevant information:%0D%0A">Clarion Support</a> with the following information:

- Your email account for the API (in the case that it is not the same as the originating email address),
- Date/time of the issue,
- The relevant section of the log file (see [Logging](https://cm.coremont.com/docs/api/sdk/details#logging) section) at the time the issue occurred, which will contain the [version](https://cm.coremont.com/docs/api/sdk/details#versioning) and correlation ID,
- What you expected to happen and what actually happened,
- Any other relevant information.

## Typical trade pricing worklow

1.  Create a trade by supplying the mandatory parameters required by the
    specific trade type using the
    **[create_trade](https://cm.coremont.com/docs/api/sdk/functions#create_trade)**
    function. The list of the mandatory parameters for each trade type
    can be found
    [here](https://cm.coremont.com/docs/api/sdk/functions#create_trade).
    The applicable values for each instrument can be obtained from the
    Clarion UI line **Pricer** screen.
2.  Identify the set of price metrics of interest. The
    [metrics](https://cm.coremont.com/docs/api/sdk/functions#metrics)
    function can be used to get a list of all supported metrics.
3.  Get the list of price metrics for the trade for current or series of
    historical dates using the
    **[price](https://cm.coremont.com/docs/api/sdk/functions#price)**
    function.

In [ ]:
from dotenv import load_dotenv
load_dotenv();

In [ ]:
# Ensure the SDK is properly installed
import clarion

In [ ]:
# Optional. 
from IPython.display import display, HTML

In [ ]:
# Ensure that the latest version of the SDK is installed  (v2.0.0 onwards - market_mode is mandatory)
clarion.version()

In [ ]:
# Obtain the authentication token. 
# Please note that token is refreshed automatically so you typically dont have to invoke this function more than once a day
clarion.server_login()

## List of the metrics that can be retrieved for price endpoint and available config

In [ ]:
# Optional. Can be used to print all rows from metrics response.
import pandas as pd
pd.set_option('display.max_colwidth', None)   #<-- this gets rid of any width size restrictions
pd.set_option('display.max_rows', None)  #<-- this gets rid of any restrictions on the num of rows displayed
pd.set_option('display.max_columns', None)  #<-- this gets rid of any restrictions on the num of columns displayed

In [ ]:
# See what metrics and options for config are available
clarion.price_metrics()

In [ ]:
# Create a list of metrics in form clarion.metric('label', 'metricType', 'assetClass', {config})
# 'label' here is an arbitrary string that can be used to identify the metric
m1 = clarion.metric('PresentValue', 'PresentValue', 'Any')

m2 = clarion.metric('Price', 'Price', 'Fx', {'Unit': 'PercentLocal'}) 
m3 = clarion.metric('AtmVol','Volatility','Fx',{'Type': 'ATM'})
m4 = clarion.metric('StrikeVol','Volatility','Fx',{'Type': 'Strike'})
m5 = clarion.metric('T+1Carry','Carry','Fx',{'Period': '1', 'StartBumpDays': '1'})

m6 = clarion.metric('FwdRate', 'Forward', 'Rates')
m7 = clarion.metric('StrikeVol', 'Volatility', 'Rates',{'Type': 'Strike', 'Period': 'Daily'})

m8 = clarion.metric('EqDelta', 'Delta', 'Equity', {'VolSpotDynamic': 'StickyStrike'})
m9 = clarion.metric('EqPrice', 'Price', 'Equity', {'Unit': 'Local'})

fx_metrics = [m1,m2,m3,m4,m5]
rates_metrics = [m1,m6,m7]
eq_metrics = [m1,m8,m9]

## Find which products are supported and required parameters

In [ ]:
instrument_types = clarion.price.instrument_types()
print(instrument_types)

In [ ]:
inst_type_params = clarion.price.instrument_type_params('FxVanilla')
print(inst_type_params)

## Pricing a Vanilla Option

Create a vanilla option trade by passing the following parameters to the **create_trade** function

| Name            | Type | Description                           | Mandatory? | Sample value                       |
|-------------------|--------|:---------------------|------------|-------------|
| id              | str  | An arbitrary identifier for the trade | Y          | fxVanilla_1                         |
| trade_type      | str  | Trade Type                            | Y          | ‘fxVanilla’                    |
| instrument_data | dict | Dictionary of instrument data pairs   | Y          | {‘CurrencyPair’: ‘USDMXN’ |
|                 |      |                                       |            | ‘Expiry’: ‘3m’             |
|                 |      |                                       |            | ‘Strike’: ‘21’                      |
|                 |      |                                       |            | ‘OptionType’: ‘Call’}                |

The values for the various parameters for the **instrument_data** dict above can be obtained from the line pricer. Here is the mapping:

| SDK Param          | UI Name        |
|--------------------|:---------------|
| CurrencyPair       | CcyPair        |
| Expiry             | Expiry         |
| Strike             | Strike         |
| OptionType         | Type           |

Get the list of **price_metrics** using **clarion.metrics** function. For this example we use the following metrics:
    -   ForwardPrice
    -   Price

Price the trade by passing the list of metrics, trade object, and the start and end date (for historical pricing only) to the **clarion.price** function.

### Full Python Code - FX Option pricing

In [ ]:
#create trade
fx_vanilla_1 = clarion.create_trade('fxVanilla_1', 'fxVanilla',{'CurrencyPair': 'USDMXN',
                                                                'Expiry': '3m',
                                                                'Strike': 'f',              
                                                                'OptionType': 'Call'})
fx_trades = [fx_vanilla_1] 

In [ ]:
#live pricing returning specified fx metrics
response = clarion.price.run(fx_metrics, fx_trades, market_mode='live')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

In [ ]:
# historic pricing returning specified fx metrics, note historic pricing based on London 4pm close
response = clarion.price.run(fx_metrics, fx_trades, start_date="2021-06-30",end_date="2022-06-30", market_mode='reval')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

### Full Python Code - Fx derivative pricing with helper functions

There are helper functions for creating FX trades. The details are [here] (https://cm.coremont.com/docs/api/sdk/functions#helper-functions). Code for pricing
 - **vanilla option**
 - **forward** 
 - **digi** 
using helper functions is given below.

In [ ]:
fx_vanilla_2 = clarion.create_fx_vanilla('fx_vanilla_2', 'EURUSD', '1m', 'atms', 'Call', 300)                               
fx_digi_1 = clarion.create_fx_digi('fx_digi_1', 'EURUSD', '1M', 's', 'Call', 'EUR', 300)
fx_forward_1 = clarion.create_fx_forward('fx_forward_1', 'EURUSD', '1M', 's', 300)
response = clarion.price.run(fx_metrics, [fx_vanilla_2, fx_digi_1, fx_forward_1], market_mode='live')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

### Full Python Code - Fx Reverse Knockout pricing

In [ ]:
fx_rko_1 = clarion.create_trade('fx_rko_1', 'fxReverseKnockOut',{'CurrencyPair': 'USDMXN',
                                                                'Expiry': '3m',
                                                                'Strike': '21',              
                                                                'OptionType': 'Call',
                                                                'BarrierUp': '26'})
response = clarion.price.run(fx_metrics, fx_rko_1, start_date="2022-03-01",end_date="2022-03-16", market_mode='reval')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

### Full Python Code - Rates swaption pricing

In [ ]:
rates_swaption = clarion.create_trade('swaption_1', 'ratesSwaption', {'InstrumentName': 'EURIBOR A 3M',
                                                                 'Currency': 'EUR',
                                                                 'Expiry': '3-May-24',
                                                                 'Notional': 100000,
                                                                 'Tenor':'1y',
                                                                 'Strike': '2.425',
                                                                 'SwaptionType': 'Rec',
                                                                 'CollateralCurrency': 'USD',
                                                                 'SettleType': 'Cash Price',
                                                                 'PaySettle': 'Fwd'})

response = clarion.price.run(rates_metrics, rates_swaption, market_mode='live')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

In [ ]:
rates_swaption = clarion.create_trade('swaption_1', 'ratesSwaption', {'InstrumentName': 'BBSW Q 3M',
                                                                 'Currency': 'AUD',
                                                                 'Expiry': '10-Mar-23',
                                                                 'Notional': 100000000,
                                                                 'Tenor':'6M',
                                                                 'Strike': '0.2',
                                                                 'SwaptionType': 'Pay',
                                                                 'CollateralCurrency': 'USD',
                                                                 'SettleType': 'Physical Cleared LCH',
                                                                 'PaySettle': 'Spot'})

response = clarion.price.run(rates_metrics, rates_swaption,market_mode='live')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

### Full Python Code - Equity future option pricing

In [ ]:
equity_futOpt = clarion.create_trade('id2','equityFutureOption',{'InstrumentName':'AS51 SFE',
                                                         'UnderlyingExpiry':'Sep23',
                                                         'ContractType':'OTC', 
                                                         'Expiry':'2023-09-21',
                                                         'Strike':'5500',
                                                         'OptionType':'C',
                                                         'ExerciseType':'A',
                                                         'Quantity':100})
response = clarion.price.run(eq_metrics, equity_futOpt, start_date="2022-06-01",end_date="2022-09-01",market_mode='reval')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

## Retrieve Exchange close pricing
#### Exchange close pricing is available back up to **4w history** from todays date

In [ ]:
rates_STIRFut = clarion.create_trade('stir1', 'ratesShortTermFuture', {'InstrumentName':'EURIBOR 3M ICE',
                                                                       'Currency':'EUR',
                                                                       'Tenor':'H23'})

In [ ]:
m1 = clarion.metric('FwdRate', 'Forward', 'Rates')
response = clarion.price.run([m1], rates_STIRFut, start_date='2022-09-01', end_date='2022-09-05',market_mode='reval',settings={'market_name':'exchange'})
response['DFRAME']